This notebook cleans up several CSVs and joins Reviews and Kaggle Product Dataset together.

IDs are standardized here as well

NULL or NaN values are set to "NULL" as MySQL Workbench recognizes it as empty values.

Other than data cleaning, several columns are also removed

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/admin/Downloads/sephora_website_dataset-unique.csv')

In [6]:
df.at[0, 'ingredients']

'Arancia di Capri Eau de Toilette: Alcohol Denat.- Water- Fragrance- Limonene- Linalool- Ethylhexyl Methoxycinnamate- Citral- Butyl Methoxydibenzoylmethane- Ethylhexyl Salicylate- BHT- Geraniol- Benzyl Alcohol- Citronellol- Tocopherol.Bergamotto di Calabria Eau de Toilette:Alcohol Denat.- Fragrance- Water- Limonene- Ethylhexyl Methoxycinnamate- Linalool- Citral- Butyl Methoxydibenzoylmethane- Ethylhexyl Salicylate- Alpha-Isomethyl Ionone- Bht- Citronellol- Geraniol. Fico di Amalfi Eau de Toilette: Alcohol Denat.- Fragrance- Water- Limonene- Ethylhexyl Methoxycinnamate- Linalool- Alpha-Isomethyl Ionone- Hexyl Cinnamal- Butyl Methoxydibenzoylmethane- Ethylhexylsalicylate- Bht- Citronellol- Hydroxycitronellal- Coumarin- Citral- Geraniol- Amyl Cinnamal- Benzyl Salicylate- Butylphenyl Methylpropional- Benzyl Alcohol- Eugenol- Benzyl Benzoate- Disodium EDTA.Mirto di Panarea Eau de Toilette:Alcohol Denat.- Fragrance- Water- Limonene- Linalool- Butylphenyl Methylpropional- Hexyl Cinnamal- Citr

In [7]:
df['ingredients'] = df['ingredients'].str.split('- ')
df2 = df.explode('ingredients')

In [11]:
df2.loc[0]['ingredients']

0     Arancia di Capri Eau de Toilette: Alcohol Denat.
0                                                Water
0                                            Fragrance
0                                             Limonene
0                                             Linalool
                           ...                        
0                                   Hydroxycitronellal
0                                          Citronellol
0                                       Benzyl Alcohol
0    Tocopherol.Acqua di Parma product ingredient l...
0    please read the ingredient list on the packagi...
Name: ingredients, Length: 80, dtype: object

In [204]:
prod = pd.read_csv('/Users/admin/Downloads/facial_products.csv')
prod_review = pd.read_csv('reviews2.csv')
prod_merge = pd.read_csv('products2.csv')
review_add = pd.read_csv('review_add.csv')
addition = pd.read_csv('addition.csv')

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [205]:
prod = prod[['brand','category','name','size','rating','love','price','value_price','URL','details','how_to_use','ingredients']]
prod.columns = [x if x != 'URL' else 'url' for x in prod.columns.to_list()]

In [206]:
prod['product_id'] = prod['url'].str.split('grid:').str[1].str.upper()

In [207]:
prod_review.columns = ['user_nickname', 'rating', 'review_text', 'context_data_values','product_id','author_id','submission_time','is_recommended','content_locale','submission_id']
prod_review = prod_review[['submission_id','product_id','rating','review_text','submission_time','content_locale','is_recommended','context_data_values']]

review_add.columns = ['user_nickname', 'rating', 'review_text', 'context_data_values','product_id','author_id','submission_time','is_recommended','content_locale','submission_id']
review_add = review_add[['submission_id','product_id','rating','review_text','submission_time','content_locale','is_recommended','context_data_values']]

prod_review = pd.concat([prod_review, review_add])

In [208]:
prod_merge.columns = ['description','product_id','image_url','product_name','review_statistics','total_reviews']

addition = addition[['Description', 'Id', 'ImageUrl', 'Name', 'ReviewStatistics','TotalReviewCount']]
addition.columns = ['description','product_id','image_url','product_name','review_statistics','total_reviews']

prod_merge = pd.concat([prod_merge, addition])

In [209]:
altmerged = prod.merge(prod_merge, how = 'left', left_on = 'product_id', right_on = 'product_id')
altmerged2 = prod.merge(prod_merge, how = 'left', left_on = 'name', right_on = 'product_name')

altmerged.dropna(thresh=14)
merged = altmerged.fillna(altmerged2)

In [347]:
altmerged.drop_duplicates('product_id').fillna('NULL').to_csv('review_product.csv', index = False)
prod_review.drop_duplicates().to_csv('review_review.csv', index = False)

In [213]:
merged.query('not total_reviews > 0')

,brand,category,name,size,rating,love,price,value_price,url,details,how_to_use,ingredients,product_id,description,image_url,product_name,review_statistics,total_reviews
65,AMOREPACIFIC,BB & CC Cream,Color Control Cushion Compact Broad Spectrum S...,1.05 oz,4.0,30900,60.0,60.0,https://www.sephora.com/product/color-control-...,Which skin type is it good for?✔ Normal✔ Oily✔...,Suggested Usage:-Using puff applicator- press ...,-Ethylhexyl Methoxycinnamate: 7%\n -Titanium ...,P378121,NaN,NaN,NaN,NaN,NaN
94,Anastasia Beverly Hills,Eyebrow,Brow Powder Duo,0.03 oz/ 2 x 0.8 g,4.5,143000,23.0,23.0,https://www.sephora.com/product/brow-powder-du...,What it is: A two-shade brow powder compact th...,Suggested Usage:-Use short- light strokes to f...,All shades contain:Talc- Boron Nitride- Nylon-...,P69300,NaN,NaN,NaN,NaN,NaN
97,Anastasia Beverly Hills,Eyebrow,Perfect Brow Pencil,0.034 oz/ 0.85 g,4.5,47900,23.0,23.0,https://www.sephora.com/product/perfect-brow-p...,What it is:\nA dual-ended brow pencil for appl...,no instructions,C10-18 Triglycerides- Hydrogenated Vegetable O...,P69305,NaN,NaN,NaN,NaN,NaN
106,Anastasia Beverly Hills,Contour,Contour Kit,6 pans x 0.11 oz/ 3.12 g,4.0,247300,40.0,40.0,https://www.sephora.com/product/contour-kit-P3...,What it is:\nA sleek palette with six shades: ...,Precautions:-Discontinue use if signs of rash ...,Java- Vanilla- Fawn- Havana- and Banana:\nTal...,P386335,NaN,NaN,NaN,NaN,NaN
109,Anastasia Beverly Hills,Eyebrow,Tinted Brow Gel,0.32 oz/ 9 g,4.0,78400,22.0,22.0,https://www.sephora.com/product/tinted-brow-ge...,What it is:A tinted brow gel.What it is formul...,Suggested Usage:-For a natural- highlighting e...,Water- Alcohol Denat.- Vp/Va Copolymer- Mica- ...,P187202,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,Yves Saint Laurent,Lipstick,Volupté Tint-in-Balm,0.12 oz,4.5,42600,34.0,34.0,https://www.sephora.com/product/volupte-tint-i...,What it is: A hybrid lipstick that combines no...,Suggested Usage:-Apply to clean lips or layer ...,Polybutene- Hydrogenated Polyisobutene- Diisos...,P417850,NaN,NaN,NaN,NaN,NaN
4408,Yves Saint Laurent,Face Serums,Pure Shots Light Up Brightening Serum Refill,1 oz/ 30 mL,0.0,72,70.0,70.0,https://www.sephora.com/product/yves-saint-lau...,What it is: An environmentally-conscious refi...,Suggested Usage:-To release the refill from th...,-Vitamin Cg: Helps to reduce the look of yello...,P454073,NaN,NaN,NaN,NaN,NaN
4438,SEPHORA COLLECTION,Lip Gloss,Rouge Smooth Shine Lip Crayon,0.04 oz/1.15 g,4.0,6200,12.0,12.0,https://www.sephora.com/product/rouge-smooth-s...,What it is:\n A lipstick crayon with a subtle ...,no instructions,Ricinus Communis (Castor) Seed Oil- Caprylic/C...,P439037,NaN,NaN,NaN,NaN,NaN
4440,SEPHORA COLLECTION,Lip Stain,Cream Lip Stain Liquid Lipstick,0.169 oz/ 5 mL,4.0,16900,14.0,14.0,https://www.sephora.com/product/cream-lip-stai...,What it is:\nA silky- long-lasting lip stain t...,Suggested Usage:-Apply directly to lips. -Star...,-Avocado Oil: Provides comfort and nutrition ...,P436320,NaN,NaN,NaN,NaN,NaN


In [348]:
altmerged

,brand,category,name,size,rating,love,price,value_price,url,details,how_to_use,ingredients,product_id,description,image_url,product_name,review_statistics,total_reviews
0,AERIN,Lip Balm & Treatment,Rose Lip Conditioner,0.34 oz/ 10 mL,4.5,17000,30.0,30.0,https://www.sephora.com/product/rose-lip-condi...,What it is: A nurturing lip conditioner with ...,no instructions,Polybutene; Hydrogenated Polyisobutene; Dextri...,P398965,What it is: A nourishing lip balm with a calm...,https://www.sephora.com/productimages/sku/s173...,Rose Lip Conditioner,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",506.0
1,Aether Beauty,Eye Palettes,Solstice Eyeshadow Palette,no size,5.0,6100,58.0,58.0,https://www.sephora.com/product/summer-solstic...,What it is: A palette of 12 golden hour shades...,Suggested Usage:-Use your favorite brushes to ...,-Organic Coconut Oil: Creates a buttery smoot...,P444401,What it is: A palette of 12 golden hour shades...,https://www.sephora.com/productimages/sku/s218...,Solstice Eyeshadow Palette,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",182.0
2,Aether Beauty,Eye Palettes,Rose Quartz Crystal Gemstone Eyeshadow Palette,no size,5.0,8200,58.0,58.0,https://www.sephora.com/product/rose-quartz-cr...,What it is: A palette with 12 shades infused w...,Suggested Usage:-Use your favorite brushes to ...,-Organic and Fair Trade** Coconut Oil: Creates...,P441803,NaN,NaN,NaN,NaN,NaN
3,Aether Beauty,Highlighter,Supernova Crushed Diamond Highlighter,0.37 fl oz/ 10.5 g,5.0,3900,38.0,38.0,https://www.sephora.com/product/aether-beauty-...,What it is: A super-soft- ethereal glow highli...,Suggested Usage:-Can be used on cheeks- eyes- ...,-Ethically Sourced Diamond Powder: Helps to p...,P454768,NaN,NaN,NaN,NaN,NaN
4,Aether Beauty,Cheek Palettes,Crystal Charged Cheek Palette,0.3 oz/ 9 g,5.0,1000,44.0,44.0,https://www.sephora.com/product/aether-beauty-...,What it is: A blush palette that comes with tw...,no instructions,-Rose Hip Oil: Provides visibly radiant- glow...,P457815,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4462,SEPHORA COLLECTION,Face Primer,Beauty Amplifier Afterglow Primer & Luminizer,no size,4.0,23700,16.0,16.0,https://www.sephora.com/product/beauty-amplifi...,What it is:\nA luminizing gel primer that can ...,Suggested Usage:\n-Apply a small amount of pri...,Water- Cyclopentasiloxane- Butylene Glycol- Mi...,P403326,What it is:\nA luminizing gel primer that can ...,https://www.sephora.com/productimages/sku/s176...,Beauty Amplifier Afterglow Primer & Luminizer,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",608.0
4463,SEPHORA COLLECTION,Eye Palettes,Winter Magic Eyeshadow Palette,8 x 0.04 oz/ 1.3 g,4.0,51800,10.0,10.0,https://www.sephora.com/product/winter-magic-e...,What it is:\n \nA travel-size palette of eight...,no instructions,unknown,P422611,What it is:\n \nA travel-size palette of eight...,https://www.sephora.com/productimages/sku/s192...,Winter Magic Eyeshadow Palette,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",159.0
4464,SEPHORA COLLECTION,Face Masks,The Rose Gold Mask,no size,2.0,6200,6.0,6.0,https://www.sephora.com/product/the-rose-gold-...,What it is: A limited-edition- nurturing and h...,Suggested Usage:-Unfold the mask.-Apply the ma...,-Rose Quartz Extract: Hydrates dry skin. Aqua...,P450138,"What it is: A limited-edition, nurturing and h...",https://www.sephora.com/productimages/sku/s220...,The Rose Gold Mask,"{'FeaturedReviewCount': 0, 'RecommendedCount':...",21.0
4465,SEPHORA COLLECTION,Lip Sets,Give Me Some Sugar Colorful Gloss Balm Set,3 x 0.32 oz/ 9 g,0.0,266,15.0,27.0,https://www.sephora.com/product/sephora-collec...,What it is: A set of three bestselling Colorfu...,Suggested Usage:-Apply directly to lips using ...,Colorful Gloss Balm Wanderlust: Hydrogenated P...,P457436,What it is: A set of three bestselling Colorfu...,https://www.sephora.com/productimages/sku/s229...,Give Me Some Sugar Colorful Gloss Balm Set,"{'FeaturedReviewCoun

In [351]:
van = pd.read_csv('product_ingredients.csv')

In [357]:
corr = pd.read_csv('product_id_correction.csv')

In [358]:
display(van.head(2))
display(corr.head(2))

,product_id,ingredient_id
0,2218774,1
1,2218774,2


,id,pid
0,1738640,P398965
1,2189975,P444401


In [364]:
vancor = van.merge(corr, how = 'left', left_on ='product_id', right_on ='id')[['pid', 'ingredient_id']].query('pid == pid')

In [373]:
vancor.columns = ['product_id', 'ingredient_id']

In [369]:
vancor.to_csv('product_ingredients_pid.csv', index = False)

In [378]:
vancor['product_id'].str.len().max()

11